In [2]:
m=[1 2 3; 4 5 6; 5 6 1; 2 3 7]

4×3 Array{Int64,2}:
 1  2  3
 4  5  6
 5  6  1
 2  3  7

In [3]:
goldcost=[0 1 3; 1 3 6; 5 8 12; 10 12 13]

4×3 Array{Int64,2}:
  0   1   3
  1   3   6
  5   8  12
 10  12  13

In [4]:
#0 = ner
#1 = höger
golddir=[9 1 1; 0 1 1; 0 1 1; 0 0 1]

4×3 Array{Int64,2}:
 9  1  1
 0  1  1
 0  1  1
 0  0  1

In [19]:
function hej(m)
    Y=size(m,1)
    X=size(m,2)
    dir=ones(Int,(Y,X))
    cost=zeros(Int,(Y,X))
    for x=1:X,y=1:Y
        if x>1
            cost[y,x]=cost[y,x-1]+m[y,x-1]
            dir[y,x]  = y==1 ? 1 : dir[y,x-1]
        end
        if y>1
            t_cost=cost[y-1,x]+m[y-1,x]
            if t_cost<cost[y,x] || x==1
                cost[y,x]=t_cost
                dir[y,x] = x==1 ? 0 : dir[y-1,x]
            end
        end
    end
    dir[1,1]=9
    return cost, dir
end

display.(hej(m))

4×3 Array{Int64,2}:
  0   1   3
  1   3   6
  5   8  12
 10  12  13

4×3 Array{Int64,2}:
 9  1  1
 0  1  1
 0  1  1
 0  0  1

(nothing, nothing)

In [22]:
display.((goldcost,golddir))

4×3 Array{Int64,2}:
  0   1   3
  1   3   6
  5   8  12
 10  12  13

4×3 Array{Int64,2}:
 9  1  1
 0  1  1
 0  1  1
 0  0  1

(nothing, nothing)

In [25]:
?circshift

search: circshift circshift!



```
circshift(A, shifts)
```

Circularly shift the data in an array. The second argument is a vector giving the amount to shift in each dimension.

```jldoctest
julia> b = reshape(collect(1:16), (4,4))
4×4 Array{Int64,2}:
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16

julia> circshift(b, (0,2))
4×4 Array{Int64,2}:
  9  13  1  5
 10  14  2  6
 11  15  3  7
 12  16  4  8

julia> circshift(b, (-1,0))
4×4 Array{Int64,2}:
 2  6  10  14
 3  7  11  15
 4  8  12  16
 1  5   9  13
```

See also [`circshift!`](@ref).


In [29]:
?flipdim

search: flipdim



```
flipdim(A, d::Integer)
```

Reverse `A` in dimension `d`.

```jldoctest
julia> b = [1 2; 3 4]
2×2 Array{Int64,2}:
 1  2
 3  4

julia> flipdim(b,2)
2×2 Array{Int64,2}:
 2  1
 4  3
```


In [33]:
min.(m,m)

4×3 Array{Int64,2}:
 1  2  3
 4  5  6
 5  6  1
 2  3  7

In [138]:
function replace!(x, KV::Pair{Int,Int}...)
    for (k,v) in KV
        x[x.==k] = v
    end
    x
end

replace! (generic function with 1 method)

In [139]:
q1(m) = flipdim(circshift(m, (-1,0)), 1)
q2(m) = q1(q3(m))
q3(m) = flipdim(circshift(m, (0,-1)), 2)
q4(m) = m

iq1(m,d) = q1(m), replace!(d, 0 => 2)
iq2(m,d) = q2(m), replace!(d, 0 => 2, 1 => 3)
iq3(m,d) = q3(m), replace!(d, 1 => 3)
iq4(m,d) = q4(m), d

iq4 (generic function with 2 methods)

In [140]:
blaj(ab) = ab[1]+ab[2]
(2,3) |> blaj

5

In [141]:
q4(m)

4×3 Array{Int64,2}:
 1  2  3
 4  5  6
 5  6  1
 2  3  7

In [142]:
iq3(q3(m))

4×3 Array{Int64,2}:
 1  2  3
 4  5  6
 5  6  1
 2  3  7

In [143]:
m[12]

7

In [144]:
m

4×3 Array{Int64,2}:
 1  2  3
 4  5  6
 5  6  1
 2  3  7

In [145]:
findmax(m)

(7, 12)

In [147]:
function travelcost(M, origin)
    m = circshift(M, (1-origin[1], 1-origin[2]))
    expand(f) = x -> f(x...)
    c1,d1 = m |> q1 |> hej |> expand(iq1)
    c2,d2 = m |> q2 |> hej |> expand(iq2)
    c3,d3 = m |> q3 |> hej |> expand(iq3)
    c4,d4 = m |> q4 |> hej |> expand(iq4)
    #r = findmin.(c1,c2,c3,c4)
    #circshift(r, (origin[1]-1, origin[2]-1))
    c1,d1
end

travelcost(m, [2,2])

([0 5 11; 10 16 13; 7 10 11; 5 7 10], [9 1 1; 2 2 2; 2 2 2; 2 2 2])

In [148]:
m

4×3 Array{Int64,2}:
 1  2  3
 4  5  6
 5  6  1
 2  3  7

In [103]:
# M is halite amount
function score(M, ship, shipyard, threshold)
    mining = max.(0, M - threshold)
    leftovers = min.(mining, threshold)
    ignoreshipyard = M[shipyard...]
    #travel = travelcost(M, ship) + travelcost(M, shipyard) + leftovers - ignoreshipyard
    #return mining - 0.1*travel
    travel = travelcost(0.1*M, ship) + travelcost(0.1*M, shipyard) + 0.1*leftovers - 0.1*ignoreshipyard
    return mining - travel
end

score (generic function with 1 method)

In [106]:
score(HALITE, [2,2], [1,1],1)

4×3 Array{Float64,2}:
  2.9  13.9  21.9
 33.9  46.9  50.9
 37.9  48.9  -9.1
 10.9  19.9  56.9

In [105]:
HALITE=m*10

4×3 Array{Int64,2}:
 10  20  30
 40  50  60
 50  60  10
 20  30  70